In [45]:
TaskObject = "{\"TaskInstanceId\":1,\"TaskMasterId\":2,\"TaskStatus\":\"InProgress\",\"TaskType\":\"TestTask Type Name\",\"Enabled\":1,\"ExecutionUid\":\"8448eabb-9ba4-4779-865b-29e973431273\",\"NumberOfRetries\":0,\"DegreeOfCopyParallelism\":1,\"KeyVaultBaseUrl\":\"https://ark-stg-kv-ads-irud.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\",\"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-1,\"EngineName\":\"ark-stg-adf-ads-irud\",\"SystemType\":\"Datafactory\",\"ResourceGroup\":\"dlzdev04\",\"SubscriptionId\":\"ed1206e0-17c7-4bc2-ad4b-f8d4dab9284f\",\"ADFPipeline\":\"GPL_AzureSqlTable_NA_AzureBlobFS_Parquet_Azure\",\"EngineJson\":\"{}\",\"TaskDatafactoryIR\":\"Azure\",\"JsonProperties\":{}},\"Source\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\",\"Type\":\"Parquet\"},\"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\",\"Type\":\"Parquet\"}}"


StatementMeta(arkstgsynspads, 5, 1, Finished, Available)

In [29]:
import random
import json
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

session_id = random.randint(0,1000000)
#invalid source
##TaskObject = "{\"TaskInstanceId\":1,\"TaskMasterId\":2,\"TaskStatus\":\"InProgress\",\"TaskType\":\"TestTask Type Name\",\"Enabled\":1,\"ExecutionUid\":\"8448eabb-9ba4-4779-865b-29e973431273\",\"NumberOfRetries\":0,\"DegreeOfCopyParallelism\":1,\"KeyVaultBaseUrl\":\"https://ark-stg-kv-ads-irud.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\",\"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-1,\"EngineName\":\"ark-stg-adf-ads-irud\",\"SystemType\":\"Datafactory\",\"ResourceGroup\":\"dlzdev04\",\"SubscriptionId\":\"ed1206e0-17c7-4bc2-ad4b-f8d4dab9284f\",\"ADFPipeline\":\"GPL_AzureSqlTable_NA_AzureBlobFS_Parquet_Azure\",\"EngineJson\":\"{}\",\"TaskDatafactoryIR\":\"Azure\",\"JsonProperties\":{}},\"Source\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\"},\"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\",\"Type\":\"Parquet\"}}"
#valid source
##TaskObject = "{\"TaskInstanceId\":1,\"TaskMasterId\":2,\"TaskStatus\":\"InProgress\",\"TaskType\":\"TestTask Type Name\",\"Enabled\":1,\"ExecutionUid\":\"8448eabb-9ba4-4779-865b-29e973431273\",\"NumberOfRetries\":0,\"DegreeOfCopyParallelism\":1,\"KeyVaultBaseUrl\":\"https://ark-stg-kv-ads-irud.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\",\"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-1,\"EngineName\":\"ark-stg-adf-ads-irud\",\"SystemType\":\"Datafactory\",\"ResourceGroup\":\"dlzdev04\",\"SubscriptionId\":\"ed1206e0-17c7-4bc2-ad4b-f8d4dab9284f\",\"ADFPipeline\":\"GPL_AzureSqlTable_NA_AzureBlobFS_Parquet_Azure\",\"EngineJson\":\"{}\",\"TaskDatafactoryIR\":\"Azure\",\"JsonProperties\":{}},\"Source\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\",\"Type\":\"Parquet\"},\"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\",\"Type\":\"Parquet\"}}"
TaskDict = {}
OutputDict = {}
TaskObjectJson = json.loads(TaskObject)

##we want to delete EngineJson as it causes issues when converting back to a json and it is not needed as its properties are within JsonProperties as children
try:
    del TaskObjectJson['ExecutionEngine']['EngineJson']
except:
    print("No EngineJson Found")


testStrings = ['Json', 'Parquet', 'Csv', 'Delta']
try:
    contains = any([test in TaskObjectJson['Source']['Type'] for test in testStrings])
except:
    contains = False


if(contains):
    Source = TaskObjectJson['Source']['System']['Container'] + "@" + TaskObjectJson['Source']['System']['SystemServer'].replace("https://","") + "/"
    Source = Source + TaskObjectJson['Source']['Instance']['SourceRelativePath'] + "/" + TaskObjectJson['Source']['DataFileName']
    Source = Source.replace('//', '/')
    Source = "abfss://" + Source

else:
    Source = "invalid"


try:
    contains = any([test in TaskObjectJson['Target']['Type'] for test in testStrings])
except:
    contains = False

if(contains):
    Target = TaskObjectJson['Target']['System']['Container'] + "@" + TaskObjectJson['Target']['System']['SystemServer'].replace("https://","") + "/"
    Target = Target + TaskObjectJson['Target']['Instance']['TargetRelativePath'] + "/" + TaskObjectJson['Target']['DataFileName']
    Target = Target.replace('//', '/')
    Target = "abfss://" + Target


else:
    Target = "invalid"
##df = spark.read.load(Source, format=SourceDT)


SourceHttpPath = TaskObjectJson['Source']['System']['Container'] + "/" + TaskObjectJson['Source']['System']['SystemServer'].replace("https://","") + "/"
SourceHttpPath = SourceHttpPath + TaskObjectJson['Source']['Instance']['SourceRelativePath'] + "/" + TaskObjectJson['Source']['DataFileName']
SourceHttpPath = SourceHttpPath.replace('//', '/')
SourceHttpPath = "https://" + SourceHttpPath

TargetHttpPath = TaskObjectJson['Target']['System']['Container'] + "/" + TaskObjectJson['Target']['System']['SystemServer'].replace("https://","") + "/"
TargetHttpPath = TargetHttpPath + TaskObjectJson['Target']['Instance']['TargetRelativePath'] + "/" + TaskObjectJson['Target']['DataFileName']
TargetHttpPath = TargetHttpPath.replace('//', '/')
TargetHttpPath = "https://" + TargetHttpPath




OutputDict['SourceHttpPath'] = SourceHttpPath
OutputDict['TargetHttpPath'] = TargetHttpPath

TaskDict['Source'] = Source
TaskDict['Target'] = Target
print(TaskDict)


StatementMeta(arkstgsynspads, 14, 2, Finished, Available)

{'Source': 'invalid', 'Target': 'abfss://datalakelanding@arkstgdlsadsirudadsl.dfs.core.windows.net/TestFile.parquet'}

In [31]:
Output = ""
for key,value in TaskDict.items():
    if value != 'invalid':
        df = spark.read.load(value, format=TaskObjectJson[key]['Type'])
        col = []
        for column in df.dtypes:
            text = {}
            text['name'] = column[0]
            text['type'] = column[1]
            col.append(text)
        OutputDict[key+'Columns'] = col
    else:
        OutputDict[key+'Columns'] = 'Invalid'
OutputDict['TaskObject'] = TaskObjectJson

Output = json.dumps(OutputDict)


StatementMeta(arkstgsynspads, 14, 4, Finished, Available)

{"SourceHttpPath": "https://datalakelanding/arkstgdlsadsirudadsl.dfs.core.windows.net/TestFile.parquet", "TargetHttpPath": "https://datalakelanding/arkstgdlsadsirudadsl.dfs.core.windows.net/TestFile.parquet", "SourceColumns": "Invalid", "TargetColumns": [{"name": "CustomerID", "type": "int"}, {"name": "NameStyle", "type": "boolean"}, {"name": "Title", "type": "string"}, {"name": "FirstName", "type": "string"}, {"name": "MiddleName", "type": "string"}, {"name": "LastName", "type": "string"}, {"name": "Suffix", "type": "string"}, {"name": "CompanyName", "type": "string"}, {"name": "SalesPerson", "type": "string"}, {"name": "EmailAddress", "type": "string"}, {"name": "Phone", "type": "string"}, {"name": "PasswordHash", "type": "string"}, {"name": "PasswordSalt", "type": "string"}, {"name": "rowguid", "type": "string"}, {"name": "ModifiedDate", "type": "timestamp"}], "TaskObject": {"TaskInstanceId": 1, "TaskMasterId": 2, "TaskStatus": "InProgress", "TaskType": "TestTask Type Name", "Enable

In [ ]:
mssparkutils.notebook.exit(Output) 

StatementMeta(, , , Cancelled, )